<a href="https://colab.research.google.com/github/ayeshasGithub/hello-world/blob/master/workflow_feeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xmltodict
!pip install redis

In [ ]:
#jupyter backup
import requests
import redis
import xml.etree.ElementTree as ET
import math
import xmltodict
import json
from collections import OrderedDict
from datetime import date, datetime
import datetime


#links for 6 workflow feeds
url_mShift = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=moU39Gbxtz9IhQO0gUy@EQAKbJcR374vb@IVb6kWwH0=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1"# 3 record
url_mJob="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GIcqmjbvPvCFX3QNIsJbTNa99MyIUuvAFUyKPTBcs9g=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" #has zero records
url_mStation="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GN3jGNVF03thDKOMppdZf/hjLH5vss2SgBlEkNaZvtE=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" #20 records
url_mZone="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=LYZEOIUPmsEILn4DoGwhlGw0VaYZ6Yh8@bEKrGOlN6A=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" # 2 records
url_mLine="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XWPuVgYIhUVEZXRsr6gAFq2BPeTCdlLJxRBpvg07WLM=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" # 1 record
url_mAsset="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=/y0R6s3pWwReO8a/3ef9tmtO8RAT/U/AKFfmPAD3egA=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1"

url={'asset': url_mAsset, 'shift': url_mShift , 'job': url_mJob, 'station': url_mStation, 'zone': url_mZone, 'line': url_mLine}

#fields to extract from the feeds
keys_to_extract= { 'asset' : ["linelink", "zonelink", "stationlink","assetname", "industry","typeofasset","modelnumber","assetmanufacturer","plancycletime"] ,
                  'station' : ['fmsstationname', 'fmsstationlinelink', 'fmsstationzonelink','fmsstationplannedmnt', 'fmsstationplannedcyctim', 'fmsstationjobsplanned', 'fmsstationtimetoent', 'fmsstationtimetopalm'],
                  'zone' : ['fmszonename', 'fmszonelinelinking', 'fmszoneplannedmnt', 'fmszoneplannedcyctim', 'fmszonejobsplanned'],
                  'shift' : ['fmsschedulename', 'fmsschedulelinelink', 'fmsschedulestarttime', 'fmsscheduleendtime','fmsschedulebreakstart','fmsschedulebreakend','fmsschedulelunchstart','fmsschedulelunchend','fmsscheduledaysnew'],
                  'line' : ['fmslinelinename', 'fmslineplannedcycletime', 'fmslinejobsplanned', 'fmslineplannedmaintenac'],
                  'job' : ['fmsjobname', 'fmsjoblinelink', 'fmsjobsshiftlink', 'fmsjobsstyle' ]}

#convert field names
keys_to_convert={ 'asset' : {'linelink':'line', 'zonelink':'zone', 'stationlink':'station','assetname':'asset', 'industry':'industry','typeofasset':'type','modelnumber':'model_number','assetmanufacturer':'manufacturer','plancycletime':'planned_cycle_time' },
                 

                  'station' : {'fmsstationname':'station', 'fmsstationlinelink':'line', 'fmsstationzonelink':'zone','fmsstationplannedmnt':'next_maintenance', 
                               'fmsstationplannedcyctim':'planned_cycle_time', 'fmsstationjobsplanned':'planned_jobs', 'fmsstationtimetoent':'planned_op_time_to_enter', 'fmsstationtimetopalm':'planned_op_time_to_palm'},
                 
                  'zone' : {'fmszonename':'zone', 'fmszonelinelinking':'line', 'fmszoneplannedmnt':'next_maintenance', 'fmszoneplannedcyctim':'planned_cycle_time', 'fmszonejobsplanned':'planned_jobs'},
                 
                  'shift' : {'fmsschedulename':'shift', 'fmsschedulelinelink':'line',  'fmsschedulestarttime':'shift_start', 'fmsscheduleendtime':'shift_end','fmsschedulebreakstart':'break_start',
                             'fmsschedulebreakend':'break_end','fmsschedulelunchstart':'lunch_start','fmsschedulelunchend':'lunch_end','fmsscheduledaysnew':'days_active'},
                 

                  'line' : {'fmslinelinename':'line', 'fmslineplannedcycletime':'planned_cycle_time', 'fmslinejobsplanned':'planned_jobs', 'fmslineplannedmaintenac':'next_maintenance'},
                 

                  'job' : {'fmsjobname':'job', 'fmsjoblinelink':'line', 'fmsjobsshiftlink':'shift', 'fmsjobsstyle':'style'}}
                 

#initialize redis host
r=redis.Redis(host="192.168.2.11", port=6379, db=0)



def convert_keys(a_subset,keys_to_extract_test,create_key,level):
    new_keys=keys_to_convert[level]#new_keys is a dictionary with old_key as key and new_key as value
    a_subset=dict([(new_keys.get(k), v) for k, v in a_subset.items()])
    return a_subset
  
def remove_redundant(a_subset,create_key,rid):
    a_subset_new=a_subset.copy()
    if rid==1:
        key_to_remove_list=create_key[0:-1]#make a list without the last key since last key is the name
        [a_subset_new.pop(key) for key in key_to_remove_list ]#redundant removed
    return a_subset_new


def fix_datetime(a_subset_new):
    from datetime import date, datetime
    date_str=a_subset_new['next_maintenance']
    format_str = '%m/%d/%Y' # The format
    datetime_obj = datetime.strptime(date_str, format_str)
    a_subset_new['next_maintenance']= datetime_obj
    return a_subset_new

def make_shift_tuple(a_subset_new):
    #keys to drop after tuples are made
    key_to_remove_listy=['shift_start','shift_end','break_start','break_end','lunch_start','lunch_end']
    format_str='%Y-%m-%d %H:%M %p'
    str1='%Y-%m-%d %H:%M %S'
    str2='%Y-%m-%d %I:%M %p'
    
        
    #make tuple of schedule
    if 'pm' in str(a_subset_new['shift_start']) and 'am' in str(a_subset_new['shift_end']):
        a_subset_new['shift_start']=str(datetime.date.today())+' ' +a_subset_new['shift_start']
        a_subset_new['shift_end']=str(datetime.date.today()+datetime.timedelta(days=1))+' ' +a_subset_new['shift_end']
    else:
        a_subset_new['shift_start']=str(datetime.date.today())+' ' +a_subset_new['shift_start']
        a_subset_new['shift_end']=str(datetime.date.today())+' ' +a_subset_new['shift_end']
        
    a_subset_new['shift_schedule']=(a_subset_new['shift_start'],a_subset_new['shift_end'])
    

    #make tuple of break
    if 'pm' in str(a_subset_new['break_start']) and 'am' in str(a_subset_new['break_end']):
        a_subset_new['break_start']=str(datetime.date.today())+' ' +a_subset_new['break_start']
        a_subset_new['break_end']=str(datetime.date.today()+datetime.timedelta(days=1))+' ' +a_subset_new['break_end']
    else:
        a_subset_new['break_start']=str(datetime.date.today())+' ' +a_subset_new['break_start']
        a_subset_new['break_end']=str(datetime.date.today())+' ' +a_subset_new['break_end']

    a_subset_new['break']=(a_subset_new['break_start'],a_subset_new['break_end'])

    #make tuple of lunch
    if 'pm' in str(a_subset_new['lunch_start']) and 'am' in str(a_subset_new['lunch_end']):
        a_subset_new['lunch_start']=str(datetime.date.today())+' ' +a_subset_new['break_start']
        a_subset_new['lunch_end']=str(datetime.date.today()+datetime.timedelta(days=1))+' ' +a_subset_new['lunch_end']
    else:
        a_subset_new['lunch_start']=str(datetime.date.today())+' ' +a_subset_new['lunch_start']
        a_subset_new['lunch_end']=str(datetime.date.today())+' ' +a_subset_new['lunch_end']

    a_subset_new['lunch']=(a_subset_new['lunch_start'],a_subset_new['lunch_end'])

    #drop keys
    [a_subset_new.pop(key) for key in key_to_remove_listy ]#redundant removed

    return a_subset_new






rid=1#remove redundant keys

def get_all(url_level,keys_to_extract_test,level,create_key,dicty):
    response = requests.get(url_level)#(url['asset'])#,headers=headers
    xml=response.text
    workflow_dict=xmltodict.parse(xml)
    new_keys=keys_to_convert[level]
    create_key = [new_keys[x] for x in create_key]#convert the key using new keys defined in keys_to_convert

    for key, value in workflow_dict.items():#looks like 1 key 1 value
    
        #print(key)#prints root

        for key2, value2 in value.items():
            
            #print(key2)#prints response
            dic_outer=json.loads(json.dumps(value2))
            fetched_records=dic_outer['@fetchedrecords']
            total_records=dic_outer['@totalrecords']
            pages=math.ceil(int(total_records)/int(fetched_records))


            dataset_value=dic_outer['NewDataSet']
            total_dics=len(dataset_value['Table1'])#dataset_value['Table1'] ===> it's a list
            print('total_records',total_dics)

            for i in range(total_dics):#total_dics == total_records

                dict_i=dataset_value['Table1'][i] #type(dataset_value['Table1'][0])====>it's a dict
                #print(dict_i)

                a_subset = {keyu: dict_i[keyu] for keyu in keys_to_extract_test} #extract necessary key from asset dictionary, keys_to_extract_asset
                a_subset = dict((k, str(v).lower()) for k, v in a_subset .items()) #values to lower case
                a_subset = convert_keys(a_subset,keys_to_extract_test, create_key, level)


                #a_subset=convert_keys(a_subset,keys_to_extract_test,level)#change the key names
                if level=='line':
                    a_subset=fix_datetime(a_subset)   
                    dicty[a_subset[create_key[0]]]=a_subset

                if level=='zone':
                    a_subset_new=remove_redundant(a_subset,create_key,rid)
                    a_subset_new=fix_datetime(a_subset_new)                    
                    if 'zones' not in dicty[a_subset[create_key[0]]]:
                        dicty[a_subset[create_key[0]]]['zones']={a_subset[create_key[1]]:a_subset_new}
                    else:
                        dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]=a_subset_new


                if level=='shift':
                    a_subset[create_key[1]]= a_subset[create_key[1]].replace(" ","_")#remove space from shift: shift 1==> shift_1
                    a_subset=make_shift_tuple(a_subset)
                    a_subset_new=remove_redundant(a_subset,create_key,rid)

                    if 'shifts' not in dicty[a_subset[create_key[0]]]:
                        dicty[a_subset[create_key[0]]]['shifts']={a_subset[create_key[1]]:a_subset_new}
                    else:
                        dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]=a_subset_new


                if level=='job':
                    a_subset[create_key[1]]= a_subset[create_key[1]].replace(" ","_")#remove space from shift: shift 1==> shift_1
                    a_subset_new=remove_redundant(a_subset,create_key,rid)
                    if 'jobs' not in dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]:
                        dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]['jobs']={a_subset[create_key[2]]:a_subset_new}

                    else:
                        dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]['jobs'][a_subset[create_key[2]]]=a_subset_new




                if level=='station':
                    a_subset_new=remove_redundant(a_subset,create_key,rid)
                    a_subset_new=fix_datetime(a_subset_new)   
                    if 'stations' not in dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]:
                        dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations']={a_subset[create_key[2]]:a_subset_new}

                    else:
                        dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]=a_subset_new



                if level=='asset':
                    a_subset_new=remove_redundant(a_subset,create_key,rid)
                    if 'assets' not in dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]:
                        dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]['assets']={a_subset[create_key[3]]:a_subset_new}
                    else:
                        dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]['assets'][a_subset[create_key[3]]]=a_subset_new



    return dicty

def create_workflow_dic():

    level='line'
    line_key=['fmslinelinename']
    dicty={}
    keys_extract=keys_to_extract[level]
    dicty=get_all(url[level],keys_extract,level,line_key,dicty)
    #print(dicty)



    level='shift'
    shift_key=['fmsschedulelinelink','fmsschedulename' ]
    keys_extract=keys_to_extract[level]
    dicty=get_all(url[level],keys_extract,level,shift_key,dicty)
    #print(dicty)


    level='job'
    job_key=['fmsjoblinelink', 'fmsjobsshiftlink', 'fmsjobname']#line+shift+job
    keys_extract=keys_to_extract[level]
    dicty=get_all(url[level],keys_extract,level,job_key,dicty)#change key name
    #print(dicty)



    level='zone'
    zone_key=['fmszonelinelinking','fmszonename']
    keys_extract=keys_to_extract[level]
    dicty=get_all(url[level],keys_extract,level,zone_key,dicty)
    #print(dicty)


    level='station'
    station_key=['fmsstationlinelink', 'fmsstationzonelink', 'fmsstationname']
    keys_extract=keys_to_extract[level]
    dicty=get_all(url[level],keys_extract,level,station_key,dicty)
    #print(dicty)

    level='asset'
    asset_key=['linelink', 'zonelink', 'stationlink','assetname']
    keys_extract=keys_to_extract[level]
    dicty=get_all(url[level],keys_extract,level,asset_key,dicty)
    #print(dicty)
    #'''
    workflow_lines={}
    workflow_lines['lines']=dicty

    #print (json.dumps(workflow_lines, indent=1, default=str))
    '''
    r_val=json.dumps(workflow_lines, default=str)
    r.set('workflow_lines',r_val)#UNCOMMENT
    #'''

import time
from random import random
wa
while True:
  print('creating dic ..')
  create_workflow_dic()

  time.sleep(wait_time)




creating dic ..
total_records 2
total_records 6
total_records 2
total_records 4
total_records 40
total_records 47
creating dic ..
total_records 2
total_records 6
total_records 2
total_records 4
total_records 40


KeyboardInterrupt: ignored

In [ ]:
#from jupyter notebook
import requests
import redis
import xml.etree.ElementTree as ET
import math
import xmltodict
import json
from collections import OrderedDict
from datetime import date, datetime
import datetime


#links for 6 workflow feeds
url_mShift = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=moU39Gbxtz9IhQO0gUy@EQAKbJcR374vb@IVb6kWwH0=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1"# 3 record
url_mJob="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GIcqmjbvPvCFX3QNIsJbTNa99MyIUuvAFUyKPTBcs9g=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" #has zero records
url_mStation="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GN3jGNVF03thDKOMppdZf/hjLH5vss2SgBlEkNaZvtE=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" #20 records
url_mZone="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=LYZEOIUPmsEILn4DoGwhlGw0VaYZ6Yh8@bEKrGOlN6A=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" # 2 records
url_mLine="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XWPuVgYIhUVEZXRsr6gAFq2BPeTCdlLJxRBpvg07WLM=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" # 1 record
url_mAsset="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=/y0R6s3pWwReO8a/3ef9tmtO8RAT/U/AKFfmPAD3egA=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1"

url={'asset': url_mAsset, 'shift': url_mShift , 'job': url_mJob, 'station': url_mStation, 'zone': url_mZone, 'line': url_mLine}

#fields to extract from the feeds
keys_to_extract= { 'asset' : ["linelink", "zonelink", "stationlink","assetname", "industry","typeofasset","modelnumber","assetmanufacturer","plancycletime"] ,
                  'station' : ['fmsstationname', 'fmsstationlinelink', 'fmsstationzonelink','fmsstationplannedmnt', 'fmsstationplannedcyctim', 'fmsstationjobsplanned', 'fmsstationtimetoent', 'fmsstationtimetopalm'],
                  'zone' : ['fmszonename', 'fmszonelinelinking', 'fmszoneplannedmnt', 'fmszoneplannedcyctim', 'fmszonejobsplanned'],
                  'shift' : ['fmsschedulename', 'fmsschedulelinelink', 'fmsschedulestarttime', 'fmsscheduleendtime','fmsschedulebreakstart','fmsschedulebreakend','fmsschedulelunchstart','fmsschedulelunchend','fmsscheduledaysnew'],
                  'line' : ['fmslinelinename', 'fmslineplannedcycletime', 'fmslinejobsplanned', 'fmslineplannedmaintenac'],
                  'job' : ['fmsjobname', 'fmsjoblinelink', 'fmsjobsshiftlink', 'fmsjobsstyle' ]}

#convert field names
keys_to_convert={ 'asset' : {'linelink':'line', 'zonelink':'zone', 'stationlink':'station','assetname':'asset', 'industry':'industry','typeofasset':'type','modelnumber':'model_number','assetmanufacturer':'manufacturer','plancycletime':'planned_cycle_time' },
                 

                  'station' : {'fmsstationname':'station', 'fmsstationlinelink':'line', 'fmsstationzonelink':'zone','fmsstationplannedmnt':'next_maintenance', 
                               'fmsstationplannedcyctim':'planned_cycle_time', 'fmsstationjobsplanned':'planned_jobs', 'fmsstationtimetoent':'planned_op_time_to_enter', 'fmsstationtimetopalm':'planned_op_time_to_palm'},
                 
                  'zone' : {'fmszonename':'zone', 'fmszonelinelinking':'line', 'fmszoneplannedmnt':'next_maintenance', 'fmszoneplannedcyctim':'planned_cycle_time', 'fmszonejobsplanned':'planned_jobs'},
                 
                  'shift' : {'fmsschedulename':'shift', 'fmsschedulelinelink':'line',  'fmsschedulestarttime':'shift_start', 'fmsscheduleendtime':'shift_end','fmsschedulebreakstart':'break_start',
                             'fmsschedulebreakend':'break_end','fmsschedulelunchstart':'lunch_start','fmsschedulelunchend':'lunch_end','fmsscheduledaysnew':'days_active'},
                 

                  'line' : {'fmslinelinename':'line', 'fmslineplannedcycletime':'planned_cycle_time', 'fmslinejobsplanned':'planned_jobs', 'fmslineplannedmaintenac':'next_maintenance'},
                 

                  'job' : {'fmsjobname':'job', 'fmsjoblinelink':'line', 'fmsjobsshiftlink':'shift', 'fmsjobsstyle':'style'}}
                 

#initialize redis host
r=redis.Redis(host="192.168.2.11", port=6379, db=0)



def convert_keys(a_subset,keys_to_extract_test,create_key,level):
    new_keys=keys_to_convert[level]#new_keys is a dictionary with old_key as key and new_key as value
    a_subset=dict([(new_keys.get(k), v) for k, v in a_subset.items()])
    return a_subset
  
def remove_redundant(a_subset,create_key,rid):
    a_subset_new=a_subset.copy()
    if rid==1:
        key_to_remove_list=create_key[0:-1]#make a list without the last key since last key is the name
        [a_subset_new.pop(key) for key in key_to_remove_list ]#redundant removed
    return a_subset_new


def fix_datetime(a_subset_new):
    from datetime import date, datetime
    date_str=a_subset_new['next_maintenance']
    format_str = '%m/%d/%Y' # The format
    datetime_obj = datetime.strptime(date_str, format_str)
    a_subset_new['next_maintenance']= datetime_obj
    return a_subset_new

def make_shift_tuple(a_subset_new):
    #keys to drop after tuples are made
    key_to_remove_listy=['shift_start','shift_end','break_start','break_end','lunch_start','lunch_end']
    format_str='%Y-%m-%d %H:%M %p'
    str1='%Y-%m-%d %H:%M %S'
    str2='%Y-%m-%d %I:%M %p'
    
        
    #make tuple of schedule
    if 'am' in str(a_subset_new['shift_start']) and 'pm' in str(a_subset_new['shift_end']):
        a_subset_new['shift_start']=str(datetime.date.today())+' ' +a_subset_new['shift_start']
        a_subset_new['shift_end']=str(datetime.date.today()+datetime.timedelta(days=1))+' ' +a_subset_new['shift_end']
    else:
        a_subset_new['shift_start']=str(datetime.date.today())+' ' +a_subset_new['shift_start']
        a_subset_new['shift_end']=str(datetime.date.today())+' ' +a_subset_new['shift_end']
        
    a_subset_new['shift_schedule']=(a_subset_new['shift_start'],a_subset_new['shift_end'])
    

    #make tuple of break
    if 'am' in str(a_subset_new['break_start']) and 'pm' in str(a_subset_new['break_end']):
        a_subset_new['break_start']=str(datetime.date.today())+' ' +a_subset_new['break_start']
        a_subset_new['break_end']=str(datetime.date.today()+datetime.timedelta(days=1))+' ' +a_subset_new['break_end']
    else:
        a_subset_new['break_start']=str(datetime.date.today())+' ' +a_subset_new['break_start']
        a_subset_new['break_end']=str(datetime.date.today())+' ' +a_subset_new['break_end']

    a_subset_new['break']=(a_subset_new['break_start'],a_subset_new['break_end'])

    #make tuple of lunch
    if 'am' in str(a_subset_new['lunch_start']) and 'pm' in str(a_subset_new['lunch_end']):
        a_subset_new['lunch_start']=str(datetime.date.today())+' ' +a_subset_new['break_start']
        a_subset_new['lunch_end']=str(datetime.date.today()+datetime.timedelta(days=1))+' ' +a_subset_new['lunch_end']
    else:
        a_subset_new['lunch_start']=str(datetime.date.today())+' ' +a_subset_new['lunch_start']
        a_subset_new['lunch_end']=str(datetime.date.today())+' ' +a_subset_new['lunch_end']

    a_subset_new['lunch']=(a_subset_new['lunch_start'],a_subset_new['lunch_end'])

    #drop keys
    [a_subset_new.pop(key) for key in key_to_remove_listy ]#redundant removed

    return a_subset_new






rid=1#remove redundant keys

def get_all(url_level,keys_to_extract_test,level,create_key,dicty):
    response = requests.get(url_level)#(url['asset'])#,headers=headers
    xml=response.text
    workflow_dict=xmltodict.parse(xml)
    new_keys=keys_to_convert[level]
    create_key = [new_keys[x] for x in create_key]#convert the key using new keys defined in keys_to_convert

    for key, value in workflow_dict.items():#looks like 1 key 1 value
    
        #print(key)#prints root

        for key2, value2 in value.items():
            
            #print(key2)#prints response
            dic_outer=json.loads(json.dumps(value2))
            fetched_records=dic_outer['@fetchedrecords']
            total_records=dic_outer['@totalrecords']
            pages=math.ceil(int(total_records)/int(fetched_records))


            dataset_value=dic_outer['NewDataSet']
            total_dics=len(dataset_value['Table1'])#dataset_value['Table1'] ===> it's a list
            print('total_records',total_dics)

            for i in range(total_dics):#total_dics == total_records

                dict_i=dataset_value['Table1'][i] #type(dataset_value['Table1'][0])====>it's a dict
                #print(dict_i)

                a_subset = {keyu: dict_i[keyu] for keyu in keys_to_extract_test} #extract necessary key from asset dictionary, keys_to_extract_asset
                a_subset = dict((k, str(v).lower()) for k, v in a_subset .items()) #values to lower case
                a_subset = convert_keys(a_subset,keys_to_extract_test, create_key, level)


                #a_subset=convert_keys(a_subset,keys_to_extract_test,level)#change the key names
                if level=='line':
                    a_subset=fix_datetime(a_subset)   
                    dicty[a_subset[create_key[0]]]=a_subset

                if level=='zone':
                    a_subset_new=remove_redundant(a_subset,create_key,rid)
                    a_subset_new=fix_datetime(a_subset_new)                    
                    if 'zones' not in dicty[a_subset[create_key[0]]]:
                        dicty[a_subset[create_key[0]]]['zones']={a_subset[create_key[1]]:a_subset_new}
                    else:
                        dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]=a_subset_new


                if level=='shift':
                    a_subset[create_key[1]]= a_subset[create_key[1]].replace(" ","_")#remove space from shift: shift 1==> shift_1
                    a_subset=make_shift_tuple(a_subset)
                    a_subset_new=remove_redundant(a_subset,create_key,rid)

                    if 'shifts' not in dicty[a_subset[create_key[0]]]:
                        dicty[a_subset[create_key[0]]]['shifts']={a_subset[create_key[1]]:a_subset_new}
                    else:
                        dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]=a_subset_new


                if level=='job':
                    a_subset[create_key[1]]= a_subset[create_key[1]].replace(" ","_")#remove space from shift: shift 1==> shift_1
                    a_subset_new=remove_redundant(a_subset,create_key,rid)
                    if 'jobs' not in dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]:
                        dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]['jobs']={a_subset[create_key[2]]:a_subset_new}

                    else:
                        dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]['jobs'][a_subset[create_key[2]]]=a_subset_new




                if level=='station':
                    a_subset_new=remove_redundant(a_subset,create_key,rid)
                    a_subset_new=fix_datetime(a_subset_new)   
                    if 'stations' not in dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]:
                        dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations']={a_subset[create_key[2]]:a_subset_new}

                    else:
                        dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]=a_subset_new



                if level=='asset':
                    a_subset_new=remove_redundant(a_subset,create_key,rid)
                    if 'assets' not in dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]:
                        dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]['assets']={a_subset[create_key[3]]:a_subset_new}
                    else:
                        dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]['assets'][a_subset[create_key[3]]]=a_subset_new



    return dicty

      
level='line'
line_key=['fmslinelinename']
dicty={}
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,line_key,dicty)
#print(dicty)



level='shift'
shift_key=['fmsschedulelinelink','fmsschedulename' ]
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,shift_key,dicty)
#print(dicty)


level='job'
job_key=['fmsjoblinelink', 'fmsjobsshiftlink', 'fmsjobname']#line+shift+job
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,job_key,dicty)#change key name
#print(dicty)



level='zone'
zone_key=['fmszonelinelinking','fmszonename']
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,zone_key,dicty)
#print(dicty)


level='station'
station_key=['fmsstationlinelink', 'fmsstationzonelink', 'fmsstationname']
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,station_key,dicty)
#print(dicty)

level='asset'
asset_key=['linelink', 'zonelink', 'stationlink','assetname']
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,asset_key,dicty)
#print(dicty)
#'''
workflow_lines={}
workflow_lines['lines']=dicty

print (json.dumps(workflow_lines, indent=1, default=str))
'''
r_val=json.dumps(workflow_lines, default=str)
r.set('workflow_lines',r_val)#UNCOMMENT
#'''






total_records 2
total_records 6
total_records 2
total_records 4
total_records 40
total_records 47
{
 "lines": {
  "rw43": {
   "line": "rw43",
   "planned_cycle_time": "248",
   "planned_jobs": "10",
   "next_maintenance": "2020-07-31 00:00:00",
   "shifts": {
    "shift_3": {
     "shift": "shift_3",
     "days_active": "monday!tuesday!wednesday!thursday!friday",
     "shift_schedule": [
      "2020-07-16 04:00 pm",
      "2020-07-16 12:00 am"
     ],
     "break": [
      "2020-07-16 06:00 am",
      "2020-07-16 06:15 am"
     ],
     "lunch": [
      "2020-07-16 08:00 pm",
      "2020-07-16 09:00 pm"
     ]
    },
    "shift_1": {
     "shift": "shift_1",
     "days_active": "monday!tuesday!wednesday!thursday!friday",
     "shift_schedule": [
      "2020-07-16 12:00 am",
      "2020-07-16 08:00 am"
     ],
     "break": [
      "2020-07-16 02:00 am",
      "2020-07-16 02:15 am"
     ],
     "lunch": [
      "2020-07-16 04:00 am",
      "2020-07-16 05:00 am"
     ],
     "jobs": {
  

"\nr_val=json.dumps(workflow_lines, default=str)\nr.set('workflow_lines',r_val)#UNCOMMENT\n#"

In [ ]:
import requests
import redis
import xml.etree.ElementTree as ET
import math
import xmltodict
import json
from collections import OrderedDict
from datetime import date, datetime


#links for 6 workflow feeds
url_mShift = "https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=moU39Gbxtz9IhQO0gUy@EQAKbJcR374vb@IVb6kWwH0=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1"# 3 record
url_mJob="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GIcqmjbvPvCFX3QNIsJbTNa99MyIUuvAFUyKPTBcs9g=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" #has zero records
url_mStation="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=GN3jGNVF03thDKOMppdZf/hjLH5vss2SgBlEkNaZvtE=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" #20 records
url_mZone="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=LYZEOIUPmsEILn4DoGwhlGw0VaYZ6Yh8@bEKrGOlN6A=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" # 2 records
url_mLine="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=XWPuVgYIhUVEZXRsr6gAFq2BPeTCdlLJxRBpvg07WLM=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1" # 1 record
url_mAsset="https://helio-ent-ice.c2m.net/Ice.svc/GetData?apikey=CSWQpToX4KLCwODDd3QCmBzak8/DYJ&feedID=/y0R6s3pWwReO8a/3ef9tmtO8RAT/U/AKFfmPAD3egA=&parameterType=LISTDATA&filtervalue=2000&filtertype=totalnoofrecords&startrecord=1"

url={'asset': url_mAsset, 'shift': url_mShift , 'job': url_mJob, 'station': url_mStation, 'zone': url_mZone, 'line': url_mLine}

#fields to extract from the feeds
keys_to_extract= { 'asset' : ["linelink", "zonelink", "stationlink","assetname", "industry","typeofasset","modelnumber","assetmanufacturer","plancycletime"] ,
                  'station' : ['fmsstationname', 'fmsstationlinelink', 'fmsstationzonelink','fmsstationplannedmnt', 'fmsstationplannedcyctim', 'fmsstationjobsplanned', 'fmsstationtimetoent', 'fmsstationtimetopalm'],
                  'zone' : ['fmszonename', 'fmszonelinelinking', 'fmszoneplannedmnt', 'fmszoneplannedcyctim', 'fmszonejobsplanned'],
                  'shift' : ['fmsschedulename', 'fmsschedulelinelink', 'fmsschedulestarttime', 'fmsscheduleendtime','fmsschedulebreakstart','fmsschedulebreakend','fmsschedulelunchstart','fmsschedulelunchend','fmsscheduledaysnew'],
                  'line' : ['fmslinelinename', 'fmslineplannedcycletime', 'fmslinejobsplanned', 'fmslineplannedmaintenac'],
                  'job' : ['fmsjobname', 'fmsjoblinelink', 'fmsjobsshiftlink', 'fmsjobsstyle' ]}

#convert field names
keys_to_convert={ 'asset' : {'linelink':'line', 'zonelink':'zone', 'stationlink':'station','assetname':'asset', 'industry':'industry','typeofasset':'type','modelnumber':'model_number','assetmanufacturer':'manufacturer','plancycletime':'planned_cycle_time' },
                 

                  'station' : {'fmsstationname':'station', 'fmsstationlinelink':'line', 'fmsstationzonelink':'zone','fmsstationplannedmnt':'next_maintenance', 
                               'fmsstationplannedcyctim':'planned_cycle_time', 'fmsstationjobsplanned':'planned_jobs', 'fmsstationtimetoent':'planned_op_time_to_enter', 'fmsstationtimetopalm':'planned_op_time_to_palm'},
                 
                  'zone' : {'fmszonename':'zone', 'fmszonelinelinking':'line', 'fmszoneplannedmnt':'next_maintenance', 'fmszoneplannedcyctim':'planned_cycle_time', 'fmszonejobsplanned':'planned_jobs'},
                 
                  'shift' : {'fmsschedulename':'shift', 'fmsschedulelinelink':'line',  'fmsschedulestarttime':'shift_start', 'fmsscheduleendtime':'shift_end','fmsschedulebreakstart':'break_start',
                             'fmsschedulebreakend':'break_end','fmsschedulelunchstart':'lunch_start','fmsschedulelunchend':'lunch_end','fmsscheduledaysnew':'days_active'},
                 

                  'line' : {'fmslinelinename':'line', 'fmslineplannedcycletime':'planned_cycle_time', 'fmslinejobsplanned':'planned_jobs', 'fmslineplannedmaintenac':'next_maintenance'},
                 

                  'job' : {'fmsjobname':'job', 'fmsjoblinelink':'line', 'fmsjobsshiftlink':'shift', 'fmsjobsstyle':'style'}}
                 

#initialize redis host
r=redis.Redis(host="192.168.2.11", port=6379, db=0)



def convert_keys(a_subset,keys_to_extract_test,create_key,level):
  new_keys=keys_to_convert[level]#new_keys is a dictionary with old_key as key and new_key as value
  a_subset=dict([(new_keys.get(k), v) for k, v in a_subset.items()])
  return a_subset
  
def remove_redundant(a_subset,create_key,rid):
  a_subset_new=a_subset.copy()
  if rid==1:
    key_to_remove_list=create_key[0:-1]#make a list without the last key since last key is the name
    [a_subset_new.pop(key) for key in key_to_remove_list ]#redundant removed
  return a_subset_new


def fix_datetime(a_subset_new):
  date_str=a_subset_new['next_maintenance']
  format_str = '%m/%d/%Y' # The format
  datetime_obj = datetime.strptime(date_str, format_str)
  a_subset_new['next_maintenance']= datetime_obj
  return a_subset_new

def make_shift_tuple(a_subset_new):
  #keys to drop after tuples are made
  key_to_remove_listy=['shift_start','shift_end','break_start','break_end','lunch_start','lunch_end']
  format_str='%H:%M %p'
  for ky in key_to_remove_listy:
    time_str=a_subset_new[ky]
    a_subset_new[ky] = datetime.strptime(time_str, format_str)
  #make tuple of schedule
  a_subset_new['shift_schedule']=(a_subset_new['shift_start'],a_subset_new['shift_end'])

  #make tuple of break
  a_subset_new['break']=(a_subset_new['break_start'],a_subset_new['break_end'])

  #make tuple of lunch
  a_subset_new['lunch']=(a_subset_new['lunch_start'],a_subset_new['lunch_end'])

  #drop keys
  [a_subset_new.pop(key) for key in key_to_remove_listy ]#redundant removed

  return a_subset_new






rid=1#remove redundant keys

def get_all(url_level,keys_to_extract_test,level,create_key,dicty):
  response = requests.get(url_level)#(url['asset'])#,headers=headers
  xml=response.text
  workflow_dict=xmltodict.parse(xml)
  new_keys=keys_to_convert[level]
  create_key = [new_keys[x] for x in create_key]#convert the key using new keys defined in keys_to_convert

  for key, value in workflow_dict.items():#looks like 1 key 1 value
    #print(key)#prints root

    for key2, value2 in value.items():
      #print(key2)#prints response
      dic_outer=json.loads(json.dumps(value2))
      fetched_records=dic_outer['@fetchedrecords']
      total_records=dic_outer['@totalrecords']
      pages=math.ceil(int(total_records)/int(fetched_records))


      dataset_value=dic_outer['NewDataSet']
      total_dics=len(dataset_value['Table1'])#dataset_value['Table1'] ===> it's a list
      print('total_records',total_dics)

      for i in range(total_dics):#total_dics == total_records

        dict_i=dataset_value['Table1'][i] #type(dataset_value['Table1'][0])====>it's a dict
        #print(dict_i)

        a_subset = {keyu: dict_i[keyu] for keyu in keys_to_extract_test} #extract necessary key from asset dictionary, keys_to_extract_asset
        a_subset = dict((k, str(v).lower()) for k, v in a_subset .items()) #values to lower case
        a_subset = convert_keys(a_subset,keys_to_extract_test, create_key, level)


        #a_subset=convert_keys(a_subset,keys_to_extract_test,level)#change the key names
        if level=='line':
          a_subset=fix_datetime(a_subset)   
          dicty[a_subset[create_key[0]]]=a_subset
        
        if level=='zone':
          a_subset_new=remove_redundant(a_subset,create_key,rid)
          a_subset_new=fix_datetime(a_subset_new)                    
          if 'zones' not in dicty[a_subset[create_key[0]]]:
            dicty[a_subset[create_key[0]]]['zones']={a_subset[create_key[1]]:a_subset_new}
          else:
            dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]=a_subset_new


        if level=='shift':
          a_subset[create_key[1]]= a_subset[create_key[1]].replace(" ","_")#remove space from shift: shift 1==> shift_1
          a_subset=make_shift_tuple(a_subset)
          a_subset_new=remove_redundant(a_subset,create_key,rid)
          
          if 'shifts' not in dicty[a_subset[create_key[0]]]:
            dicty[a_subset[create_key[0]]]['shifts']={a_subset[create_key[1]]:a_subset_new}
          else:
            dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]=a_subset_new
           

        if level=='job':
          a_subset[create_key[1]]= a_subset[create_key[1]].replace(" ","_")#remove space from shift: shift 1==> shift_1
          a_subset_new=remove_redundant(a_subset,create_key,rid)
          if 'jobs' not in dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]:
            dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]['jobs']={a_subset[create_key[2]]:a_subset_new}

          else:
            dicty[a_subset[create_key[0]]]['shifts'][a_subset[create_key[1]]]['jobs'][a_subset[create_key[2]]]=a_subset_new



      
        if level=='station':
          a_subset_new=remove_redundant(a_subset,create_key,rid)
          a_subset_new=fix_datetime(a_subset_new)   
          if 'stations' not in dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]:
            dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations']={a_subset[create_key[2]]:a_subset_new}

          else:
            dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]=a_subset_new



        if level=='asset':
          a_subset_new=remove_redundant(a_subset,create_key,rid)
          if 'assets' not in dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]:
            dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]['assets']={a_subset[create_key[3]]:a_subset_new}
          else:
            dicty[a_subset[create_key[0]]]['zones'][a_subset[create_key[1]]]['stations'][a_subset[create_key[2]]]['assets'][a_subset[create_key[3]]]=a_subset_new
          

      
      return dicty

      
level='line'
line_key=['fmslinelinename']
dicty={}
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,line_key,dicty)
#print(dicty)



level='shift'
shift_key=['fmsschedulelinelink','fmsschedulename' ]
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,shift_key,dicty)
#print(dicty)


level='job'
job_key=['fmsjoblinelink', 'fmsjobsshiftlink', 'fmsjobname']#line+shift+job
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,job_key,dicty)#change key name
#print(dicty)



level='zone'
zone_key=['fmszonelinelinking','fmszonename']
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,zone_key,dicty)
#print(dicty)


level='station'
station_key=['fmsstationlinelink', 'fmsstationzonelink', 'fmsstationname']
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,station_key,dicty)
#print(dicty)

level='asset'
asset_key=['linelink', 'zonelink', 'stationlink','assetname']
keys_extract=keys_to_extract[level]
dicty=get_all(url[level],keys_extract,level,asset_key,dicty)
#print(dicty)
#'''

#print (json.dumps(dicty, indent=1, default=str))
#'''




#write workflowfeeds on redis
'''
r_val=json.dumps(dicty)
r.set('workflow_lines',r_val)#UNCOMMENT
dict_y=r.get('workflow_lines')
print (json.dumps(dict_y, indent=1, default=str))
#'''

ModuleNotFoundError: ignored